# Predicting Airport Delays

Ruth Ashford, Spencer Song, Rajiv Verma, Lana Elauria, Carolina Arriaga

MIDS W261 Section 7: Team 15

## Question Formulation

### Objective
We will be predicting which flights will be delayed at departure by more than 15 minutes. We will be making the prediction two hours in advance to the scheduled departure time. This will benefit airlines who are behind in On-Time Performance or OTP (OTP is defined as the percentage of flights having less than 15 mins of delay) for proactive cost-savings.

### Business Context
OTP is important in airlines operations management. The FAA estimates that delays cost the airlines $8.3b in 2019, increasing ~50% from 2016 ([Source](https://www.faa.gov/data_research/aviation_data_statistics/media/cost_delay_estimates.pdf)). OTP is also important for customer satisfaction. On-time flights mean less time waiting at airports, fewer missed connections, and a happy customer. Higher customer satisfaction would lead to higher loyalty for an airline. Typical OTP performance for airlines is between 70 and 80%. Airlines can improve OTP performance up to 90% to get a leg up on their rivals and attract customers. Most industry publications state that getting beyond 90% OTP will have exorbitant costs, and the benefits might not be enough to justify the outlay ([Source](https://www.oag.com/on-time-performance-airlines-airports)). Beyond a specific number, customers may not care about OTP. So, if we can find around 40-50% of flights that will be delayed with high precision and devote resources (e.g., personnel, gates) to get them out in time, it will give us the right outcome.

It would be a false positive if we mark a flight as delayed and would have departed on time. If we did this, we would have allocated costly resources on the wrong flight without any benefits and taken away those resources from other potentially delayed flights. If we mark a flight as non-delayed and would not have departed on time, it would be a false negative. In this scenario, we will miss out on an opportunity to target a flight and improve OTP. A false positive has a higher cost than a false negative and furthers our case for giving precision higher weightage.

### Primary Metric
Since we want to target airlines, we choose the F measure with a beta of 0.5 as our primary metric. This gives higher importance to precision, but we need to keep recall at 40-50% to get enough flights to change the metric significantly. The calculation for F Beta (0.5) is: $$F Beta (0.5) = \frac{(1+0.5^2) * precision * recall)}{0.5^2 * precision + recall}$$

### Sources of data 

##### Flight information 
The primary dataset for this analysis would be flight level information provided by the U.S. Department of Transportation (DOT). This dataset is publicly available and contains information about flight departures, arrivals, cancellations & diversions, and information about airports and airlines. We'll use a 109 field subset of the passenger flight's on-time performance data taken from the TranStats data collection available from the U.S. DOT. As seen in Figure 1, the number of flights per year has steadily increased, whereas the percentage of delayed flights remains relatively constant.

###### Figure 1. Number of flights per year.
<img src ='/files/shared_uploads/ruth.ashford@berkeley.edu/yearly_eda.png'>


##### Weather reports
From our personal experiences when flying, we know that weather conditions can significantly impact flight delays. For this reason, we'll also use weather data corresponding to a flight's origin airport at the time of departure and build features based upon this data. We'll use data from the National Oceanic and Atmospheric Administration repository from January 2015 to December 2019. It contains hourly and daily weather information collected from various stations worldwide. It has cloud cover, rain, snow, wind, and temperature attributes. We identified weather data by weather station id.

##### Weater stations and their neighbors 
Most airports have a weather station, but obtaining weather reports from additional nearby weather stations may give us more recent or more accurate weather information. The weather station location dataset provided the airport where a weather station is located where applicable, along with the neighboring weather stations.   

##### IATA codes

An IATA code is a unique identifier for an airport. We mapped to an IATA code to establish a shared key between the flights' and weather station data.

##### Holidays
From prior experience, we know that many flights get delayed around the holidays. To tease out this factor, we downloaded a list of all holidays from 2014 to 2019, including principal and minor holidays in the US. We rated them on a 0-2 score, with 0 being least traveled and 2 being those holidays where most Americans travel. For example, we rated Christmas and Thanksgiving as 2 and Memorial Day Weekend as a 1. 

##### Useful but unavailable datasets
We searched for a dataset that included the number of gates & passengers handled at each airport to account for congestion. However, we couldn't find any dataset that matched our requirements. Another factor that might be useful was the capacity and age of the aircraft. But this data was not publicly available, and we couldn't get it for our analysis.

### Business Constraints
Based on our objective, a prediction on if a flight will be delayed or not needs to be made two hours before the flight's planned departure. At that point, there are many unknowns like the aircraft might not have arrived from the previous leg. Since we also have to consider weather from two hours before, changes to the weather conditions are possible in two hours. All of these could lead to a weak prediction. Also, airlines may pad extra time into their schedules so that the flights don't get delayed. Padding can lead to lesser delays but reduces revenues due to lower aircraft rotation; airlines are less motivated to do it. During the timeline of our dataset, there have been a few airline mergers, notably one between Virgin and Alaska. Mergers can lead to a loss of precision because previously independent airlines' data is absent from the parent airline during prediction. These scenarios will require special handling.

## Phase Summaries
### Phase I
<b>Progress:</b> Created BLOB storage setup for saving intermediate data. Created Trello board and assigned tasks to team members. Performed EDA on airline and weather data. Decided on outcomes & we want to target. Imported external data to join airline and weather data. Cleaned column data for all datasets, timestamps merged across datasets. We have removed some features and have begun planning additional features that we want to create.  

<b>Plan:</b> Clean up external data source and focus on join implementation. We are planning on joining data on origin airport codes. Once the join is complete, we will save the results to the BLOB storage for future use. We will also try our train/test/CV split strategy on smaller dataset.  

<b>Issues:</b> Working on missing IATA codes and don’t have concrete direction yet on that fix. Airport time is in local time while weather is in UTC. We will need airporttime library installed to join these datasets. Also, limited knowledge within the team on Spark MLlib pipeline setup to build initial model.    

### Phase II
<b>Progress:</b> Split out tasks into further data cleaning, feature encoding, joining data and creating a baseline model. Weather data was explored further, determining useful columns and utilizing the quality fields to make decisions on data accuracy. Performed feature encoding on the airport data set, standardizing numberical columns and using either one hot encoding or frequency encoding for categorical data based on the number of categories. Successfully joined the 3 month airport, station and weather data together. Created a baseline logistic regression model using the airport data, performed some light feature encoding and got a precision of 0.32.   

<b>Plan:</b> Feature encoding for the weather data set. Perform more advanced feature encoding using decision tree based models to determine the splits. Perform further EDA on the weather data to determine the best way to aggregate the weather reports across neighboring stations. 

<b>Issues:</b> Some of the weather data is ambiguous, such as the 'condition' fields, need to perform more research on how to interpret this. Need to determine the best way to split the data for cross-validation when using the full data set, so as to avoid issues with seasonality. 

### Phase III
<b>Progress:</b> Scaled up tasks to full dataset. Performed [join on full dataset](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1898361324238488/command/1898361324238489) and troubleshooted new issues from scaling up (e.g., incorrectly mapped airlines). [Encoded and aggregated weather data](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/4070574709969017/command/4070574709969018), [performed EDA](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1898361324259717/command/4070574709971213), and [evaluated feature importance on full joined dataset](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1898361324247597/command/1898361324247598). Updated baseline model with weather features, evaluated performance of updated model on full joined dataset. Our model got a precision of 0.59, with an F-score of 0.42.

<b>Plan:</b> Experiment with feature selection and perform GridSearch/RandomCV to finalize features and tune hyperparameters. Explore novel ideas other than weather aggregation method, such as random forest algorithm implementation or dealing with imbalanced data through over-/under-sampling.

<b>Issues:</b> Need to research implementation of SMOTE at scale, as well as come up with novel directions to take the model. Sourcing data for any novel feature may be difficult, although we have already implemented some additional features with the data we already have, such as previous delays for a specific aircraft.

## EDA & Discussion of Challenges

### Data integrity
The datasets we were working with required a thorough cleaning before use. Weather data was incredibly challenging as it was available in custom data structures and formats unsuitable for machine learning. Based on available documentation, we joined various weather condition components to create a usable attribute related to rain, snow, and other weather elements. We considered six parts of weather: temperature, cloudiness, atmospheric pressure, precipitation, wind, and humidity. For precipitation elements like rain & snow, we thought about both the duration and accumulation. We focused only on hourly reported weather fields and disregarded daily summaries due to how much weather can change in hours.

The flights' data contained information that we may not be able to know two hours in advance of the flight departure, such as cancellation reasons or flight diversions. We excluded these columns except when we could use them for feature engineering, such as knowing the arrival time of an aircraft's last journey. 

#### Missing data and quality control
The weather data contained special codes to denote if the value for a weather observation was missing. A quality code was also present for each weather observation type. We nullified all data marked as missing or any quality code marked as 'erroneous' or 'suspect' for easier processing.

#### Nulls and Duplicates
Some of the columns contained a very high percentage of null values >95%; however, this small amount of information could be beneficial for an attribute like snow, which is a rare occurrence. Due to this, we decided not to drop columns with high volumes of null fields but instead to compute a suitable value. To determine the best value to use, we looked at the ratio of nulls in each class and compared it. We realized the proportions (~0.003% to 0.004%) were pretty similar for each class and that the ratio of null values was around 0.3%. We decided to impute a value of zero in these cases given their nature, e.g., the amount of rain.  

We considered duplicates among both the rows and columns of the datasets. For row duplicates, we defined a unique entry in the flights' data set as distinct across flight date, planned departure time, and flight path, which dropped 0.78% of the data. For column duplicates, we excluded features with the same meaning but in a different format. For example, between the "city name" and "city name abbreviated" features, we only kept one. Other examples include carrier and carrier id, destination and destination id, amongst others.

### Exploratory Data Analysis
The code for the following exploratory data analysis can be found in our team's [EDA Projects notebook](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1898361324259717/command/1858507102415333).

#### Seasonality 
We wanted to evaluate whether there were any seasonal patterns in flight delays. We hypothesized that there would be more delays in peak travel months, such as the summer vacation months of July and August since airport congestion could cause various logistic issues that pile up and lead to delays. Also, harsher weather conditions in November and December combined with holiday travel rushes could lead to a similar effect in the year's later months, with lulls in delay frequency in between peak months. To explore this hypothesis, we grouped flights by month and created a boxplot of the percentage of delayed flights per month (Figure 2). We see some seasonal patterns in the plot, matching our expectations, but the standard deviation for each month is 5% to 7%. The standard deviations of each month's delay percentage also lie between 5% and 7%. We may capture the boxplot patterns by natural variation over time. Still, we think there may be something further to explore regarding seasonal patterns of flight delays, so we included the month of the year as a feature in our predictive model.

We also plotted the percentage of flight delays for each airport each month (Figure 3) to see whether there might be geographical patterns or features associated with specific airports or regions of the country. Differences in flight delay percentages can be seen both across space and across time: there are clear clusters of airports with higher delay frequencies on both the east and west coast of America, specifically in the San Fransisco Bay, Los Angeles County, and New England, and comparing the chart for June flights to the chart for September flights reveals more significant delay percentages nationwide at the start of the summer compared to the end of the season.

###### Figure 2. Flight delays by month.
<img src='files/shared_uploads/lana.elauria@berkeley.edu/seasonality-1.png' width='450' height='20'>



###### Figure 3. Flight delays across America (June and September, 2015-2018 Airlines data).
<img src='files/shared_uploads/lana.elauria@berkeley.edu/delay_june_sep-1.png' width='1500' height='20'>

#### Holidays

We imported a holiday dataset and manually added feature importance to rank the holidays. We then mapped this importance to each flight, an ordinal indicator of the holiday status for each flight. We found this feature had minimal correlation with our target variable (a correlation value of -0.02), so we decided to omit this feature from our models.

#### Weather conditions
We expected that weather conditions such as the dew point temperature, how low the clouds were, and the amount of snowfall would correlate with whether the flight was delayed or not. Figure 4, which uses weather and flight data from 2015, shows that the average cloud height is lower when flights are delayed. We can also see many variations in the snowfall data, and we see a slight increase in snowfall when flights are delayed.  
The weather condition is categorical and very descriptive. We thought the hourly present weather condition would be a helpful field. This field is null over 70% of the time, and the topmost frequently reported weather conditions are the same across both delayed and non-delayed flights. However, we saw that delayed flights had more mist conditions than non-delayed flights. 

###### Figure 4. Weather conditions between classes (2015 Weather and Airlines data).
<img src='files/shared_uploads/ruth.ashford@berkeley.edu/weather_eda.png'>

#### Independence
Over 130 features were available in the combined flight and weather data, from which about half were numerical columns. We created a correlation matrix to understand which features correlate with each other, excluding the target variable (Figure 5). The criteria to remove a feature was a correlation below/above -0.95/0.95. The intention was to avoid multicollinearity in the logistic regression model.

###### Figure 5. Correlation matrix of features.
<img src='files/shared_uploads/caro.arriaga@berkeley.edu/heatmap_numerical.png' width='600' height='20'>

#### Flight delays during the week

Intuitively, we would think that there would be more delays on Mondays and Fridays, as most business travel happens on these days. We plotted the distribution of delays for each day of the week to confirm this hypothesis (Figure 6), which is based on 2014-2018 flight data. From the figure, we can see increases on the aggregate flight delay percentages on Mondays and Fridays.

###### Figure 6. Number of delays throughout the week (2015-2018 Airlines data).
<img src='/files/shared_uploads/ruth.ashford@berkeley.edu/week_day_delays.png'>

#### Flight delays during the day
During the day, we would expect more delays later in the day as aircrafts do more journeys and are impacted by various factors like airports, security, gates. We divided the day into five parts and plotted the percentage of delays for each time of day, using the 2014-2018 flights data (Figure 7). From the figure, we can see more delays in the latter part of the day compared to the early morning.

#### Flight delays and the distance covered by the flight
Flights have to carry different amounts of fuel depending on the distance they are traveling, which changes based on the weather conditions. Due to this, we hypothesized that longer flights might suffer from longer delays if they are waiting for the wind to calm down, for example. From Figure 7, we see that the number of flights delays is pretty consistent regardless of flight distance.

#### Flight delays by carrier
Some Airlines are better at preventing delays than others (figure 7). Hawaiian and Alaska Airlines have the highest on-time percentage in our dataset, while budget carriers JetBlue and Frontier have the lowest.

###### Figure 7. Number of delays per airline, flight distance and time of day (2015-2018 Airlines data).
<img src='/files/shared_uploads/ruth.ashford@berkeley.edu/airlines_cat_eda.png'>

### Combining flight and weather data
Weather reports from more than four hours before the planned departure time or from weather stations further away could contain helpful information, but exploding this already large dataset is very resource and time-intensive. Instead, we constructed a time window for each flight from two to four hours before the flight's scheduled departure time to combine the flight and weather information. We set the window end to two hours before a flight's scheduled departure time to avoid leakage in our time-series weather data. We also chose the start of the window by considering the average time interval between weather observations and how much the dataset would grow by joining multiple weather observations to each flight. We joined the weather data in this time window from the five closest weather stations to a flight's departure airport onto each flight, giving a complete picture of the weather conditions around each flight in our dataset. The weather reports' time and distance restraints determined both practical meaning and resource availability, including weather observations that would reflect the conditions at the airport while trying to keep the performance costs of the join as small as possible. The full join was performed in [a separate notebook](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1898361324238488/command/1898361324238489) and written to our cluster's cold blob storage for use in our aggregation and feature engineering phases.

## Feature Engineering

<!-- --ruth and carolina --spencer and lana -->

<!-- Notes:  
- ~~weather aggregation~~
- ~~transforms~~
- ~~categorical handling~~
- ~~feature selection~~
- ~~scaling/standardization~~ -->

### Weather report aggregation
After combining the flight and weather reports data, we had approximately X weather reports per flight that we needed to combine into a single set of weather conditions. We approached this in two ways, depending on if the variable consisted of numerical or categrical data. The code for these aggregations can be found in [libs/weather_aggregation](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/4070574709971748/command/4070574709971749).

##### Numerical
To take into account weather station proximity and recency of the report, we aggregated across multiple entries using an inverse distance and inverse time weighted average, excluding any null entries. For example, for a flight \\(F\\)  if we have \\(n\\) weather reports of a weather variable \\(\textbf{v} = \[v_1, ...., v_n\]\\), with corresponding weather report distance from the departure airport \\(\textbf{d} = \[d_1, ...., d_n\]\\) and time difference between weather report entry to flight departure \\(\textbf{t} = \[t_1, ...., t_n\]\\), then we calculated the weighted average for \\(\textbf{v}\\) as:  
$$vAverage = \frac{\frac{1}{t_1}\frac{1}{d_1}*v_1 + ... + \frac{1}{t_n}\frac{1}{d_n}*v_n}{\frac{1}{t_1}\frac{1}{d_1} + ... +\frac{1}{t_n}\frac{1}{d_n}} $$

##### Categorical
To combine the entries for categorical variables such as the present weather condition, we found the mode of the variable among the weather reports obtained for the flight and used this as the aggregate value. We aggregated our categorical weather features to achieve a sort of majority consensus between the weather stations around the flight's departure airport, ultimately making our model more robust to missing or messy weather data.

### Feature transformations
To achieve the best performance possible from our model, we included functions in our pipeline that would normalize and encode features to allow for better prediction when fed into a classification algorithm. The transformations differ between numerical and categorical features. We encoded the latter based on the cardinality of the features. The code is in the [libs/transform notebook](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/4070574709981865/command/4070574709981866). 

##### Numerical Features
We checked for skewed distributions of each continuous feature and applied a logarithmic transformation when required.

###### Standarization
Continuous variables were standardized when used in regression models, and we added a step in the pipeline to standardize these features according to their mean and standard deviation.

##### Categorical Features
We handled categorical features as nominal values. We used frequency encoding to convert the independent variables into an ordinal type. We also used target mean encoding for variables with over 30 categories to collapse the number of feature categories when they exceeded the number of features in our model.

###### One Hot Encoding
We limited the use of OHE to avoid an excess of dimensionality and reserved it for the weather condition feature. Based on the EDA performed, this feature was relevant for use in our training. However, its categories were largely unrelated, so it made more sense to separate them into a binary indicator rather than train the model on one feature encompassing several different weather conditions.

###### Frequency Encoding
We used a descending frequency encoding for features with less than thirty categories. The encoding created an ordinal relationship between the categories in the dataset that could contain more information than unordered categorical data. We applied the string indexer method from `pyspark.ml` for this transformation.

###### Target Mean Encoding
We decided to use target mean encoding for categorical variables with high cardinality because decision tree models cannot handle features with cardinality higher than the maximum tree depth. We decided to apply the target mean encoding instead of the frequency encoding discussed previously.

### Feature Selection
The feature selection process had two stages. First, during exploratory analysis, while working on the six-month data set, we ran a random forest model using only the categorical features to understand which had the highest importance in the model. We detected that the airline carrier, origin, and origin city market features significantly affected the result (Figure 8). The second stage of feature selection was during the hyper-parameter tuning phase. After reviewing the top-performing models, we extracted the top **_x_** features present in each of the three best models to construct a set of crucial features. Then ran a reduced model (x=9). We wanted to compare the F-scores between a more parsimonious model with our best full model (x=29). We chose not to use principal component analysis to avoid losing interpretability for this model. Figure 8 shows the first-stage categorical feature selection results.

###### Figure 8. Feature importances from first stage of feature selection.
<img src='files/shared_uploads/caro.arriaga@berkeley.edu/exploratory_cat_feat.png' width='900' height='20'>

### Time-based features
If an aircraft performed multiple flights in a day, we anticipated that given a previous flight delay, it would also impact the subsequent flight. We created a new time-based feature that indicated if the aircraft's last flight had been delayed or not. We only checked delays for flights that took off more than two hours and less than twelve hours before the departure of the subsequent flight. As shown in Figure 10, we found that 17% of delayed flights were also delayed in the previous flight. On-time flights were delayed only 5% of the time when the last flights were delayed.

###### Figure 9. Impact of previous flight delay on subsequent flights (2015-2018 Airlines data).
<img src="/files/shared_uploads/ruth.ashford@berkeley.edu/prev_flight_delay.png">

## Algorithm Exploration

<!-- - ~~custom cross validation~~
- parameter tuning / folds
- ~~over / under sampling~~
- ~~feature importance / noise considerations~~
- ~~logistic regression~~
- ~~RF~~
- ~~XGBoost~~
- ~~catboost~~
- ~~ensemble (cherry) hehe~~
- ~~Final selected model and all details~~ section below -->

Our team experimented with several classification algorithms for our final model: logistic regression, random forest, XGBoost, and Catboost. We kept a log of each model we tried in order to compare their scores and performances while choosing our final model. The model log can be found through this [link](https://docs.google.com/spreadsheets/d/189d1R7gP-ZdZgpYe524YkitWRfF4X-eAer-sgI7_M5g/edit?usp=sharing), and the models we experimented with are described in detail below.

#### Cross Validation & Grid Search

For cross validation, we used rolling windows or a Blocking Time Series split technique. We laid the training data from 2015-18 as time series. We took 9 months in each training split and 3 months in validation split. However, each training split overlapped with the previous validation split. For example, the first split was Jan 2015 - Sep 2015 for training and Oct 2015 - Dec 2015 for validation. For the second split, we used Oct 2015 - Jun 2016 for training and Jul 2016 - Sep 2016 for validation. And so on. In this manner, we were able to create 5 splits based on 4 year data.

###### Figure 10. Visualization of first 4 time series cross validation splits.
<img src='files/shared_uploads/rajiv.verma@berkeley.edu/cross_validation.png' width='800' height='18'>

We also did a custom Grid Search implementation which took the same cross validation technique so that best parameters can be found using time series cross validation.

#### Over/Under Sampling
The target classes in our dataset are highly imbalanced. The vast majority of flights in the dataset are not delayed, so that majority class for our target variable severely outweighs the minority. This introduced a heavy bias into our initial model towards non-delayed predictions, since most of the training examples were examples of non-delayed flights. To combat this bias, we introduced additional pre-processing in our data pipeline to either over-sample the minority delayed class, or under-sample the majority non-delayed class. We created algorithms that performed both under-sampling and over-sampling, since we were not sure which method would lead to a higher improvement in model performance. With each classification algorithm we experimented with, we tested model performance on the default class distribution, over-sampling, and under-sampling. With the default class distribution, our models would achieve a solid precision, around 0.6, but the recall would remain very low, 0.3 or lower. Both under-sampling and over-sampling balanced the precision and recall scores for our models, but led to approximately similar F-beta(0.5) scores as the default class distribution, all hovering around 0.4. Over-sampling tended to bring larger improvements in the F-scores of our experimental models, though, so we chose to over-sample the minority delayed class in our final model.

#### Logistic Regression

Logistic Regression was used for our baseline model. To prepare data for logistic regression, the categorical columns for encoded. For columns with less categories, we used one hot encoding while for columns with higher categories, mean encoding was used.  Grid search with custom cross validation (time series) was used to find out best parameters. The parameters we experimented with were regularization, threshold & maxIter. We experimented with oversampling and undersampling was used to improve metrics. However, both these techniques reduced precision and increased recall. So, eventually both were removed from the pipeline. Models were run on flights data, flights +weather and flights+weather+time based attributes. The last set of features gave the best score having a F beta of 0.456

#### Random Forest

Part of our gap analysis after the midterm presentation was comparing our model to other teams on the leaderboard. We noticed that many of the top-performing teams were running random forest and XGBoost classification algorithms, and they noted their precision in the 0.9 range, with around 0.8 recall and F-score. We wanted to see if a random forest classifier could achieve a similar performance with our model. We utilized PySpark's `RandomForestClassifier` class to test random forest classification on our feature set, additionally experimenting with both over-sampling and under-sampling to balance the target classes in our full joined data. The main hyperparameters tweaked in our grid search were the number of trees in the forest, and the maximum depth of those trees. Models with more trees and deeper trees outperformed those with less trees and shallow trees, but there was a performance and runtime trade-off that we needed to consider as well. Trees deeper than 15 levels took over two hours to train, and models with more than 30 trees ran into a similar issue. In the end, the best performance in both metrics and runtime were achieved by a model with 30 trees, with a maximum tree depth of 15. This model utilized over-sampling to balance the target classes. On our cross validation training set, this model achieved a precision of 0.423, a recall of 0.515, and an F-beta(0.5) score of 0.423, with a runtime of 75 minutes. On the held-out test set, this random forest model achieved a precision of 0.424, a recall of 0.486, and an F-beta(0.5) score of 0.435, with a runtime of 38 minutes. Compared to the highest-performing teams, our random forest fell short of solid predictions, but was still improved by over-sampling.

#### XGBoost
The XgboostClassifier was used to train our model on our fully selected feature set (x=29). We ran the default parameters achieving an F-beta(0.5) score of 0.427. We decided to run a random search grid tested on three of the five cross-validation splits to reduce the training time and see potential hyperparameter candidates that could surpass the F-beta score of the default model. About 20 models were run and resulted in lower values. We tested the top performers' most important features (x=9) and trained a reduced model. In most experiments, we used undersampling to handle the class imbalance except in one case where we used oversampling . The reduced model had a worse performance (F-beta score=0.304). We experimented with parameters such as *max_depth, n_estimators, reg_alpha, reg_lambda, pos_weight_score, learning_rate, and gamma* during the random search cross-validation. The model [XGB4](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/4070574710008346/command/4070574710008347) (max_depth=3, n_esitmators=100) results were precision of 0.420, recall of 0.563 and F-score of 0.427. We used these model parameters to run on the test data. The best model was XGB4, although we ran on the test XGB1 because pyspark would crash due to running out of memory in the shuffle phase, which would not happen when running the model with undersampling. The reduced model performance was significantly lower, with a decrease of 0.123 points in the F-score(0.5) compared with XGB4. We believe that the selected features are not noise but help predict delayed flights better.

#### Catboost
CatBoost has been known to gain incrementally better results than XGBoost in certain usecases, so we wanted to try it out in this project. We used the `CatboostClassifier` to train on our full set of features (29 variables). Training the model with the default Catboost hyper parameters and using undersampling of our training data gave us an average cross validation precision of 0.353, recall of 0.577 and F-score of 0.381 and took around 50 minutes to run. We then lowered the number of iterations the model performs to 10 (the default is 1000) and Catboost automatically adjusts the learning rate for us. This was much faster to run at 26 minutes and had marginally better performance to when we used 1000 iterations. We wanted to explore further hyper parameter tuning on the Catboost model but unfortunetly it doesn't work particularly well with Spark and cross validation and frequently runs into errors with cluster usage. 

#### Ensemble
We created two ensamble model using the XGB-T, LR-T and RF-T model predictions. We approached it using a hard and a soft vote (E1-hv, E2-sv, respectively). The latter resulted in the best prediction output with an F-beta(0.5)=0.507. The hard vote was a mode of the 3 predictions, while the soft vote summed the probability of the outputs and took the larger value.

#### Runtime
The runtime mean was 1:13:02 (n=15), having the minimum value at 0:18:55 (LR-T) and the maximum value at 3:43:00 (RF6). From the test sets, LR-T model was the fastest of all.

#### Table of Results

##### Train set
| Model | Feature # | Sampling |Set | Precision | Recall |F(0.5) | 
|------------|-------------|------------- | ------------- |------------- |------------- |------------- |
|XGB1 | 28 | under |train |0.402 |0.599 | 0.415 |
|XGB2 | 9 | under | train |0.282 |0.598 |0.304 |
|XGB3 | 9 | under | train |0.297 |0.62 |0.297 |
|**XGB4** | 28 | over |train |0.42 |0.563 | **0.427** |
|LR1 | 22 | none | train |0.589 |0.236 |0.447|
|**LR2** |21 |none | train |0.591 |0.233 |**0.447** |
|**RF1** | 16 | none | train |0.576 |0.264 |**0.466** |
|RF2 | 22 | none | train |**0.617** |0.169 |0.404 |
|RF3 | 22 | under | train |0.375 |0.552 |0.401 |
|RF4 | 22 | over | train |0.412 |0.515 |0.423 |
|RF5 | 22 | over | train |0.413 |0.512 |0.424 |
|RF6 | 22 | over | train |0.432 |0.479 |0.436 |
|CB1 | 28 | under | train |0.353 |0.577 | 0.381 |
|**CB2** | 28 | under | train |0.372 |0.567 | **0.395** |

##### Test set
| Model | Feature # | Sampling |Set | Precision | Recall |F(0.5) | 
|------------|-------------|------------- | ------------- |------------- |------------- |------------- |
|XGB-T | 28 | under | test |0.375 |0.599 |0.405 |
|LR-T | 21 | none | test |0.553 |0.268 | 0.456 |
|RF-T | 22 | over | test |0.424 |0.486 | 0.435 |
|E1-hv | mix | mix | test |0.491 |0.454 | 0.483 |
|**E2-sv** | mix | mix | test |0.545 |0.397 | **0.507** |

## Error Analysis 

<!-- --rajiv and whoever uses it -->

For error analysis, we marked each predicted row as a True Positive, False Positive, False Negative or True Negative. Then, we grouped on these columns and calculated aggregations of other columns. An example of the values obtained through this error analysis can be found in Figure 11, and the key results from our analysis are as follows:

- **Distance:** We expect shorter distances to be less troublesome in terms of delays and that's what we see in data. We see True Positives and False Positives as having the similar mean distance and most likely some flights with longer distance were classified as positives because of it. The negatives had lower mean distances.

- **Cloud Ceiling Height:** We expect lower cloud height to be associated with greater delays. The mean value for True Postives is clearly lower. The False Positives have a much higher value for cloud cover. From this, we conclude that cloud cover most likely didn't play as big a role in differentiating between True and False positives.

- **Scheduled Departure Time:** From EDA, we know that flights later in the day have higher delays. We find that the mean value for scheduled departure time in the True Positive group is higher, confirming our EDA and expectations. We see a big difference between the TP and FP means. From this, we can say that this variable was not as important in differentiating between True Positives and False Postives.

- **Horizontal Visibility:** The mean values for the four groups are not very different, which would mean that this attribute was not as important in our model. We do see that True Positives have lower horizontal visibility on average when compared to the other groups. Developments in flight and geolocation technology may have made this variable less important in predicting flight delays than we initially thought.

- **Wind Speed:** The classes match our expectation of higher wind speed leading to more delays. The means for True Positives and False Positives are very close, and they are higher than the mean values for the negative groups, which might mean this could be one of the key causes of misclassification.

- **Previous Depature Delay:** From our EDA, we know that previous departure delays are a good predictor of future delays. We do see that there is a large difference between the mean values of this variable from positives and negatives. As mean values for True and False Positives are similarly significantly higher than the negative means, this feature could be one of the key contributers to misclassfication between True and False Positives.

Overall, it looks like the wind speed and previous delay indicator features could be the features causing the most confusion between True Positives and False Positives in our model.

###### Figure 11. Error analysis example for a logistic regression model.
<img src='files/shared_uploads/rajiv.verma@berkeley.edu/lr_errors2.png' width='850' height='20'>

## Algorithm Implementation
For this aspect, we explored a Logistic Regression model. The experimentation was done in a [separate notebook](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/4070574709981847/command/4070574710015167).

The Logistic Cost Function is as follows: 

<img src="/files/shared_uploads/spencersong@berkeley.edu/1_P1Wu65ic5sK8Jhq8Sl_WxQ.png" width = '400' height = '20'>

<img src="/files/shared_uploads/spencersong@berkeley.edu/1_dEZxrHeNGlhfNt_JyRLpig_1.png" width = '400' height = '20'>

For application, we used a small data subset with 3,000 rows and an 80:20 train test split. For the features we only used a few features on holidays and previous delays, as the results of this implementation were on too small of a dataset to apply to our results.

This process is something that we could scale up in parallel to the full dataset in replacement of our Logistic Regression function if need be. Considerations for the gradient descent could be using a Stochastic gradient descent function to reduce runtime. We don't really see much benefit in usuing the this implementation as opposed to the Logistic package. 

For the basic model and a default learning rate of .1, here are the loss values over 40 iterations. As we can see and expect, we see a clean and gradual loss function, with poor results. With 8 cores / 32GB, the 40 iterations and plotting took 5.86 minutes. This implementation should be scalable, but we could optimize it by reducing the iteration number with a higher learning rate and with random grid search.


<img src="/files/shared_uploads/spencersong@berkeley.edu/algo/Screen_Shot_2022_04_10_at_3_08_38_PM.png" width = '800' height = '80 '>

## Final Model Selection and Business Goals
The best model was the ensemble E2-sv, which had the highest F-beta(0.5) score of 0.507. The recall was 0.397 and the precision was 0.545. The model had a 20% improvement over the baseline model.

Considering 976,000 delayed flights per year in average (year 2015-2019), we can detect 387,482 potential delayed flights and we would correctly predict 211,172 out of those. According to the FAA, 8.3b usd are spent per year on delayed flights operations, by using the soft ensemble model, airline carriers together could save about $1.795b usd per year in flight operational costs (8,504 per delayed flight).

## Conclusions

#### Successes and Business Takeaways
We started out with the objective of improving OTP for airlines by precisely predicting flight delays 2 hours in advance. We find that through the use of key datasets & right algorithms, we can help the airlines find a delayed flight with a precision of over 61%. Airlines will be able to save various organizational costs and streamline their operations with the predictions from our best model. We also find that weather attributes are key to improving prediction scores. However, we didn't see extra benefit in collecting data for larger windows and would recommend future researchers to focus on a weather window of two to three hours. Other key findings were the huge impact of prior flight delays and flights departures later in the day. Our recommendation to airlines would be to put resources to fix flight delays earlier in the day so that we can see the benefits throughout the day.

On the technical domain, the XGBoost and RandomForest models gave good results for our task, and we would recommend future researchers continue tuning these models. Our CatBoost model did not perform as expected, and we would advise future researchers against using them until major revisions are made to the underlying model code. The benefits of over-sampling and under-sampling were not seen on precision but if future research is focused on improving recall, we found that these approaches would provide significant benefits.

**Gap Analysis:** Our final model achieved a similar F-score as other groups, around 0.5 to 0.6. Logistic regression and gradient-boosted decision trees were common among other groups' work, and one other group also tried an ensemble, although their final model is an ensemble consisting of multiple logistic regression models. Most other groups also experienced a trade-off between precision and recall, and many of them have a sharp contrast between their precision and recall scores. The over-sampling algorithm implemented in our models led to more of a balance between our team's precision and recall scores, although the F-scores remain similar.

#### Challenges and Constraints
Many of the challenges our team faced stemmed from the sheer size of the dataset. Every experiment or adjustment that we made in the development process needed to be filtered through millions of rows of data, and this often took time. As such, our team faced a bottleneck while joining the full dataset: much of the work in the latter half of this project revolved around experimentation with the full joined dataset, and every bug or oversight on our part took hours to correct due to the runtime of our jobs.  We needed to take time to learn about what the weather measurements meant, and our lack of knowledge of air travel logistics limited our creativity in feature selection and model tuning. 

## Reflection

#### Future Iterations
If our team had more time to devote to iterating our model, we would take more time to explore ideas that we could not explore in full in this first iteration. Namely, we would include more features that contain information about airport congestion and seasonal patterns in flight delays, and we would take more time to develop more domain knowledge about air travel logistics and weather phenomena. We would also experiment more with graph relationships in our data, possibly including some measure of centrality of an airport as a proxy for its air traffic and congestion. We would also like to find more external sources of data so we can have a broader foundation to build our hypotheses on, encompassing logistical or mechanical delays as well as delays caused by weather conditions.

#### Our Learning Experience
Our team thought that the focus of the project on building an end-to-end pipeline for a big data machine learning project was a great learning experience. The challenges that came with working with real-world data instead of clean sample datasets illuminated much of the data engineering side of machine learning projects. Also, couching the project in a concrete business context that required us to develop domain-specific insights allowed us to collaborate like an actual team of data scientists in the industry, developing the "softer" skills that are important for a data scientist, such as resource allocation, team organization, time management, and communication of pipeline progress and challenges.



<!-- Removed lines ***

This is covered in future iterations as what we would do if we had time which reflects the same sentiment - The time and attention this large dataset required of our team had a ripple effect in the later stages of our project, leaving us with less time and attention to give to more novel ideas we wanted to explore, such as testing our pipeline on the paradigm-shift of 2020, exploring graph relationships in our data and features, or including more thorough consideration of airport congestion and traffic. Our team also faced some challenges in playing "catch-up" on the domain knowledge that this project required of us.

We can show this in successes section - Despite these difficulties, however, we were able to create a model that our team is happy with, and we learned a lot through dealing with our various challenges and obstacles.


#### Lana
- takeaways/decisions
  - PySpark debugging
  - runtime & order of operations in pipeline
  - data aggregation & bottlenecks
- wish list:
  - more time to experiment with ensemble
  - test on 2020
  - international?
- challenges
  - data aggregation & join bottleneck
  - bringing together work from different people, keeping everyone on the same page/avoiding duplicate work
  - sharing cluster resources & simultaneous jobs

### Caro
- takeaways/decisions
  - creating a pipeline start to end
  - cross validation and fine tunning
- wish list:
  - play more with graphs for feature engineering
  - better understanding on seasonality
- challenges
  - Breiman's method application
  - Getting LGBM running
  - Adding interactions
  
### Rajiv
- Fantastic project for real world data science
  - Real world data science is challenging
  - take combination of business knowledge & data analysis to improve a metric
  - Allows us to complete an end to end data science project
- Time based features were useful
- Limitations of data
  - More external data would have helped
  - we were not able to model congestion & aircraft age/capacity
  - Some data we thought was useful turned out to be not as useful
  - Need to find ways to do experimentation quicker
- Limitations of algorithm
  - In big data ML, there are less algorithms available and there's less flexibility
  - We didn't see expected benefits of tree based algorithms over LR
  
### Spencer
- more considerations with data continuity.
- Efforts to implement features and models have various costs and flexibility.
- LR goat
- Domain explanation and report behind studies of causality of delays would have been super helpful.
- Having a team 'lead' would like have boosted our productivity as a group.

### Ruth
- data sets can be cryptic and hard to understand parts of (e.g weather data), without direct access to the author of the data it can be hard to get clarifications and we have to make some assumptions 
- 
 -->

## Project Constraints

In general, the majority of our constraints boiled down to resource management. The majority of our runtimes come from training and splitting the full joined datset, so we had a major bottleneck in progressing until that full join was processed. Our decision to aggregate the weather data post-join is one that felt important to avoid information loss, but it only contributed to this bottleneck in our work. 

Along the lines of resource management, the direction of our exploration was also influenced by our time and resources. Novelty in exploring the 2020/2021 datasets was thought about too late-- we realized that the 2020 and 2021 weather data is inconsistent compared to the 2015-2019 data. Similar avenues of novelty exploration were slowed by the time consumption for minimal chances to improve our results.

The majority of our visualizations for the weather data are based on the 2015 dataset, as we had difficulty running the visualizations for the full dataset given the density of the plots for scatter plots, etc. Although there are libraries available for performing visualizations with Spark DataFrames such as Koalas, they are not particularly advanced, so only a small number of plot types are available.


External datasets were difficult to get to grips with, especially the weather data. There were a vast number of columns and a number of which the documentation for was ambigious on how it should be interpretted, such as the condition code. Given we do not have direct access to the authors of these data sets, we had to make a few assumptions about the data based on our best guess.

### Application of Course Concepts

This project was a culmination of everything we've been learning in class for the past 14 weeks. The class is solely focused on "scaling up" machine learning algorithms to be performed in parallel, at scale, and this final project was a chance for our team to apply the theory that we've learned in the asynchronous lectures and our previous homework assignments. Whereas our previous assignments were guided walkthroughs of algorithms or Hadoop/Spark structures, our model for this project was built from the ground up, and we created an end-to-end pipeline much like the ones in our assignment notebooks. Therefore, many of the considerations in our previous assignments took center stage for this project, even if they seemed peripheral in our previous work:

- Our team took concepts from our asynchonous lectures, such as Brieman's method, and implemented them in our pipeline, either in the EDA, feature creation, or model tuning phases.
- This project required an understanding of Spark's lazy evaluation in order to hit deadlines on a reasonable timeline; our team needed to know which operations were transformations and which were actions, and structure our pipeline to make full use of all transformations before calling an action, such as a `display()` or write command.
- Caching and checkpointing data was crucial in this project, as re-running certain operations could take hours, and we needed to set up checkpoints and utilize Spark's in-memory caching in our notebooks to streamline the EDA and feature creation phases.
- This class was very focused on building parallel machine learning algorithms from the ground up; while we were able (and encouraged) to use Spark packages for our classification model, we still needed to know how Spark's logistic regression, random forest, and XGBoost algorithms worked in order to optimize hyperparameters, experiment with different features, and troubleshoot errors in our pipeline and predictions.
- Much of the work for our previous assignments was spent understanding the YARN interface and the process of debugging Spark jobs through logs and DAG's. Many issues that arose during our development and testing process needed to be debugged through DataBrick's Spark logs and DAG's, so we utilized the experience from our previous projects to debug our functions and models throughout this entire final project.
- We built an algorithm from the ground up and ran it on a toy dataset, utilizing the RDD implementation of Spark mappers and reducers that we worked through in each previous assignment.
- The DataBricks cluster we were given access to for this class was an additional constraint: the consideration of the upper limits on our computational power was a constant throughout this project, and the changes in the number of workers available to our cluster were a large part of our workflow and our transition from unit testing to full-scale implementation.
- Many lectures in the class were focused on how many workers one would need to complete a job in Spark, and how adding or reducing workers would affect the runtime and performance of a model at scale, but this project was the first time in the class that we could realistically run into issues stemming from these changes.
- We used the graphframes to do some feature engineering and explored using indegree and the ratio of delayed flights given a specific origin. Due to time, we weren't able to put the feature into the model but we could easily include it as a future work.

Overall, this project was the final lesson in "scaling up" a machine learning project - we worked with large datasets and guided walkthroughs of algorithms in previous assignments, but building an end-to-end pipeline for this project required our team to see the bigger picture as well as the minute details of machine learning at scale.

# Appendix

#### Links to All Notebooks:
Data Pipeline:
- [6m Data Join](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/2866290024313870/command/2866290024313874)
- [Full Join](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1898361324238488/command/1898361324238489)
- [Test Data Join](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/4070574709997061/command/4070574709997062)

EDA:
- [EDA First Pass](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/2901002372872797/command/1038316885898966)
- [EDA Projects](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1898361324259717/command/1858507102419232)
- [Stations EDA](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1898361324239733/command/1858507102434299)

Libs/Helper Functions:
- [airlines_data](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1898361324253917/command/1898361324253918)
- [airport_codes_data](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1898361324259395/command/1898361324259438)
- [custom_cv](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1731553363140353/command/1731553363147334)
- [custom_cv_class](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1731553363147054/command/1731553363147055)
- [data_joins](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1898361324261147/command/1898361324261149)
- [error_analysis](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1858507102380710/command/1858507102380711)
- [model_helper_functions](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/4070574709988152/command/4070574709988153)
- [random_grid_builder](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1731553363137658/command/1731553363137659)
- [station_data](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1898361324253915/command/1898361324255132)
- [time_based_features](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/4070574709968966/command/4070574709968967)
- [transform](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/4070574709981865/command/4070574709981866)
- [weather_aggregation](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/4070574709971748/command/4070574709971749)
- [weather_data](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1898361324253637/command/1898361324253638)

Models:
- [Algo from scratch](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/4070574709981847/command/4070574709983677)
- [baseline_model_6m](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/3816960191356315/command/3816960191356316)
- [baseline_model_alldata](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1898361324247597/command/1898361324247598)
- [catboost_model](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/4070574710008418/command/1858507102431674)
- [ensemble_model](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1858507102408839/command/1858507102408840)
- [logistic_regression_model_alldata](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/4070574710008274/command/4070574710008275)
- [random_forest_model_alldata](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/4070574710008490/command/4070574710008491)
- [xgboost_model_alldata](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/4070574710008346/command/4070574710008347)
- [Model Log (spreadsheet)](https://docs.google.com/spreadsheets/d/189d1R7gP-ZdZgpYe524YkitWRfF4X-eAer-sgI7_M5g/edit?usp=sharing)

#### Cluster Details:
Max: 7 workers, each worker: 16GB memory, 4 cores.

thanks :)